# Galakser 

In [145]:
%%HTML
<style>
p {font-size:1.2em;
  font-family: Georgia, serif};
</style>

In [74]:
import pandas as pd

In [75]:
bøker = pd.read_csv("../../ngram_2021_coordination/coord-bok.csv", header = None)

In [76]:
bøker.columns = ['lang', 'first', 'coord', 'second', 'freq']

In [77]:
bøker[bøker['first'] == '.'].head(2)

,lang,first,coord,second,freq
8927,nno,.,og,"0,75",4
8928,nno,.,og,09.01.2010,4


In [121]:
"a".isalpha()

True

In [124]:
bøker_words = bøker[bøker['first'].map(lambda x: x.isalpha())][bøker['second'].map(lambda x: x.isalpha())]

/tmp/ipykernel_154307/342431437.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bøker_words = bøker[bøker['first'].map(lambda x: x.isalpha())][bøker['second'].map(lambda x: x.isalpha())]


In [125]:
bøker_words

,lang,first,coord,second,freq
15380,nno,A,og,Fig,4
15381,nno,A,og,Marianna,4
15382,nno,A,og,VB,4
15383,nno,A,og,alt,4
15384,nno,A,og,anten,4
...,...,...,...,...,...
4752037,nob,mer,og,mer,593743
4752038,nob,rett,og,slett,597504
4752052,nob,av,og,til,984899
4752055,nob,først,og,fremst,1637971


In [134]:
coord = bøker_words.groupby(['first', 'second']).sum()

In [80]:
from math import log

In [81]:
#coord['weight'] = coord['freq']*coord['pmi']

In a position "$x\textrm{ and }y$" the part frequencies of $x$, $x_f$, are used in computing $\frac{p(y|x)}{p(y)}$. All proportions are computed from that particular position. So the what is computed
is

1. $p(x) = \frac{x_f}{\sigma(x_f)}$ and
1. $p(y|x) = \frac{y_f^x}{\sigma(x_f)}$. 

Here $y_f^x$ is the the frequency of $y$ in the context $x$. The complete formula to be taken to pandas is then: 

$p(x|y) = \frac{x^y}{\sigma(y)}$ 

$p(y) = \frac{\sigma(y)}{t}$


In [175]:
cond = lambda x, level = 0:(coord.xs(x, level = level)/coord.xs(x, level = level).sum())

In [176]:
summation = lambda x, level = 0: coord.xs(x, level = level).sum()

In [186]:
summation('Hamsun', 0)

freq    4756
dtype: int64

In [193]:
triples = pd.DataFrame([(x, summation(x, 0)) for x in set(coord.index.get_level_values(level = 0))], columns = ['word', 'as_first'])

In [194]:
tuples_second = pd.DataFrame([(x, summation(x, 1)) for x in set(coord.index.get_level_values(level = 1))], columns = ['word', 'as_second'])

In [222]:
tuples_second = tuples_second.set_index('word')

In [224]:
tuples_second = tuples_second.fillna(0).applymap(int)

In [237]:
tuples_second.loc['i']

as_second    2809784
Name: i, dtype: int64

In [227]:
totals_second = tuples_second.sum()
totals_second

as_second    231478939
dtype: int64

In [225]:
#tuples_second.to_csv("second_position_frequencies.csv", header = True)

## Fix values and such

In [200]:
galaxy_freqs = pd.concat([triples.set_index('word'), tuples_second.set_index('word')], axis = 1)

In [208]:
galaxy_freqs = galaxy_freqs.fillna(0).applymap(int)

In [220]:
#galaxy_freqs.to_csv('galaxy_frequencies.csv', header = True)

## Compute pmi

In [219]:
galaxy_freqs.loc['spiser']

as_first     17552
as_second    42501
Name: spiser, dtype: int64

In [230]:
tuples_second.loc['spiser']

as_second    42501
Name: spiser, dtype: int64

In [250]:
pmi_series = lambda x:((coord.loc[x]/coord.loc[x].sum())['freq']/(tuples_second.loc[x]/totals_second)['as_second'])

In [258]:
pmi_series('helse').sort_values(ascending = False).head(30)

second
velferd           127.171211
trivsel           115.190268
miljø              98.109004
sykdom             86.909951
sikkerhet          80.498462
velvære            63.357053
livskvalitet       54.684102
omsorg             53.661632
sosial             53.445109
utvikling          40.983004
utdanning          35.666160
arbeidsmiljø       33.573103
sosiale            30.493664
førlighet          16.094881
for                15.204730
hygiene            15.192701
sosialsektoren     14.723568
sunnhet            14.482987
livsstil           14.182260
liv                13.701098
samfunn            13.508633
sosialfag          13.159790
et                 13.063558
en                 11.138909
forebygge          10.862240
rus                10.272816
lykke               9.948032
ernæring            9.430782
levekår             9.142085
krefter             7.746714
Name: freq, dtype: float64

In [259]:
coord.loc['helse']['freq'].sort_values(ascending = False).head(30)

second
velferd           10572
trivsel            9576
miljø              8156
sykdom             7225
sikkerhet          6692
velvære            5267
livskvalitet       4546
omsorg             4461
sosial             4443
utvikling          3407
utdanning          2965
arbeidsmiljø       2791
sosiale            2535
førlighet          1338
for                1264
hygiene            1263
sosialsektoren     1224
sunnhet            1204
livsstil           1179
liv                1139
samfunn            1123
sosialfag          1094
et                 1086
en                  926
forebygge           903
rus                 854
lykke               827
ernæring            784
levekår             760
krefter             644
Name: freq, dtype: int64

In [173]:
cond2('drikker').sort_values(by = 'freq', ascending = False).head(20)

,freq
second,
spiser,0.081893
drikker,0.081610
røyker,0.056673
er,0.041513
spiller,0.038254
snakker,0.028903
sover,0.028762
prater,0.028053
danser,0.027345


In [174]:
coord.xs('drikker', level = 0).sort_values(by = "freq", ascending = False).head(20)

,freq
second,
spiser,578
drikker,576
røyker,400
er,293
spiller,270
snakker,204
sover,203
prater,198
danser,193


In [147]:
coord.loc["smør"].sort_values(by = 'freq', ascending = False).head(50)

,freq
second,
ost,15519
sukker,10167
mel,4132
brød,2679
olje,2647
margarin,2272
egg,1903
en,1750
melk,1497


In [ ]:
def make_graph(word):
    seconds = coord.loc["